In [ ]:
import datetime, math, os, pathlib
from collections import Counter

import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.ticker import FuncFormatter
import matplotlib.dates as mdates

import paramiko
from scp import SCPClient

import sciris as scy
import covasim as cv
import covasim.parameters as cvp

import gc
import concurrent.futures

cv.git_info()['covasim']['version']

# Visualize COVASIM Outputs for Manuscript
The following code creates figures and results used in the manuscript

In [ ]:
def load_sim(i, c='C0'):
    file_name = f"results/Sim_{i}.{c}.pkl"
    try:
        sim = cv.Sim.load(f"{file_name}")
        print(f"loaded {file_name}")
        return sim
    except:
        print(f"missing {file_name}")
        return None

def extract_and_append(sims, sim):
    plot_data = {
        'time': sim.results['date'],
        'cum_diagnoses': sim.results['cum_diagnoses'],
        'new_diagnoses': sim.results['new_diagnoses'],
        'cum_severe': sim.results['cum_severe'],
        'new_severe': sim.results['new_severe'],
        'cum_deaths': sim.results['cum_deaths'],
        'new_deaths': sim.results['new_deaths'],
        # Add more variables as needed
    }
    sims.append(plot_data)

def make_sims_list(num_files=10, batch_size=50, scen_name='C0'):
    sims = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        for start in range(1, num_files + 1, batch_size):
            end = min(start + batch_size, num_files + 1)
            sims_batch = list(executor.map(lambda i: load_sim(i, c=scen_name), range(start, end)))
            for sim in sims_batch:
                if sim is not None:
                    extract_and_append(sims, sim)
            gc.collect()  # Explicitly release memory after processing each batch

    return sims

num_files = 10
batch_size = 5

plot_data_list_C0 = make_sims_list(num_files=num_files, batch_size=batch_size, scen_name='C0')

# Now you can use plot_data_list to create your plots
# Example:


In [ ]:
sim0 = load_sim(1,c="C0")
actualdata = sim0.data
actualdata[['cum_diagnoses', 'cum_severe', 'cum_deaths']] = actualdata[['cum_diagnoses', 'cum_severe', 'cum_deaths']]*7.0
print(actualdata)

In [ ]:
actual_dfs = [
    actualdata[['date', 'cum_diagnoses']],
    actualdata[['date', 'new_diagnoses']],
    actualdata[['date', 'cum_severe']],
    actualdata[['date', 'new_severe']],
    actualdata[['date', 'cum_deaths']],
    actualdata[['date', 'new_deaths']]
]

In [ ]:
def calculate_mean_var(var_series, data_list):
    # Dictionary to store mean and variance for each date
    date_statistics = {}

    # Iterate through each dictionary in data_list
    for data in data_list:
        dates = data["time"]
        values = data[var_series].values

        # Iterate through each date and corresponding value
        for date, value in zip(dates, values):
            # Calculate mean and variance for each date
            if date not in date_statistics:
                date_statistics[date] = {"values": [value]}
            else:
                date_statistics[date]["values"].append(value)

    # Arrays to store dates, means, and variances
    dates_array = []
    means_array = []
    variances_array = []

    # Populate arrays
    for date, stats in date_statistics.items():
        dates_array.append(date)
        values = np.array(stats["values"])
        means_array.append(np.mean(values))
        variances_array.append(np.var(values))

    # Convert arrays to numpy arrays if needed
    dates_array = np.array(dates_array)
    means_array = np.array(means_array)
    variances_array = np.array(variances_array)

    # Create a Pandas DataFrame
    df = pd.DataFrame({
        'Date': dates_array,
        'Mean': means_array,
        'Variance': variances_array
    })

    return df



In [ ]:
def make_df_list(plot_data_list):
    meanvar_cum_diagnoses = calculate_mean_var('cum_diagnoses', plot_data_list)
    meanvar_new_diagnoses = calculate_mean_var('new_diagnoses', plot_data_list)
    meanvar_cum_severe = calculate_mean_var('cum_severe', plot_data_list)
    meanvar_new_severe = calculate_mean_var('new_severe', plot_data_list)
    meanvar_cum_deaths = calculate_mean_var('cum_deaths', plot_data_list)
    meanvar_new_deaths = calculate_mean_var('new_deaths', plot_data_list)
    simulated_dfs = [meanvar_cum_diagnoses, meanvar_new_diagnoses, meanvar_cum_severe, meanvar_new_severe, meanvar_cum_deaths, meanvar_new_deaths]
    return simulated_dfs

In [ ]:
def custom_plot(actual_dfs, simulated_dfs, titles):
    num_plots = len(actual_dfs)
    num_rows = 3
    num_cols = 2

    plt.figure(figsize=(15, 10))

    # xtick_labels = ['Jan\n2020', 'Apr', 'Jul', 'Oct', 'Jan\n2021', 'Apr', 'Jul', 'Oct', 'Jan\n2022', 'Apr', 'Jul', 'Oct', 'Jan\n2023']

    for i, (actual_df, simulated_df, title) in enumerate(zip(actual_dfs, simulated_dfs, titles), 1):
        if title == 'Cumulative Hospitalizations':
            correct_hosp = simulated_df.Mean.loc[176]
            simulated_df['Mean'] = simulated_df['Mean'].sub(correct_hosp).clip(lower=0)

        plt.subplot(num_rows, num_cols, i)

        # Plot actual data as points with smaller markers
        plt.scatter(actual_df['date'], actual_df.iloc[:, 1:], marker='o', color='red', label=f'Actual {title} (CDC Data)', s=5)

        # Plot mean values as lines for simulated data
        plt.plot(simulated_df['Date'], simulated_df['Mean'], label='Mean (Simulated)', linestyle='-')
        # Plot filled region between upper and lower bounds of variance for simulated data
        plt.fill_between(simulated_df['Date'], simulated_df['Mean'] - np.sqrt(simulated_df['Variance']),
                          simulated_df['Mean'] + np.sqrt(simulated_df['Variance']),
                          alpha=0.2, label='Variance Bounds (Simulated)')

        plt.title(title)
        plt.xlabel('Date')
        plt.ylabel('Counts')

        # Set xticks explicitly based on the specified labels
        # plt.xticks(actual_df['date'][::len(actual_df['date']) // (len(xtick_labels) - 1)], xtick_labels)
        
        # Format xticks as month and year
        plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))        
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

        plt.legend()
        plt.grid(True)

    plt.tight_layout()
    plt.show()




In [ ]:
# Example usage
titles = ['Cumulative Diagnoses', 'New Diagnoses', 'Cumulative Hospitalizations', 'New Hospitalizations', 'Cumulative Deaths', 'New Deaths']
simulated_dfs_C0 = make_df_list(plot_data_list_C0)

# correct cumulative hospitalizations
actual_hosp = actual_dfs[2]
# actual_hosp.iloc[176:200]

In [ ]:
custom_plot(actual_dfs, simulated_dfs_C0, titles)

## Counterfactuals

In [ ]:
plot_data_list_C1 = make_sims_list(num_files=num_files, batch_size=batch_size, scen_name='C1')
plot_data_list_C2 = make_sims_list(num_files=num_files, batch_size=batch_size, scen_name='C2')
plot_data_list_C3 = make_sims_list(num_files=num_files, batch_size=batch_size, scen_name='C3')
plot_data_list_C4 = make_sims_list(num_files=num_files, batch_size=batch_size, scen_name='C4')

In [ ]:
simulated_dfs_C1 = make_df_list(plot_data_list_C1)
simulated_dfs_C2 = make_df_list(plot_data_list_C2)
simulated_dfs_C3 = make_df_list(plot_data_list_C3)
simulated_dfs_C4 = make_df_list(plot_data_list_C4)

In [ ]:
custom_plot(actual_dfs, simulated_dfs_C1, titles)


In [ ]:
custom_plot(actual_dfs, simulated_dfs_C2, titles)


In [ ]:
custom_plot(actual_dfs, simulated_dfs_C3, titles)


In [ ]:
custom_plot(actual_dfs, simulated_dfs_C4, titles)

#  Scenario Comparison

In [ ]:
# Format y-axis labels in millions for ax1 and thousands for ax1_twin
import matplotlib.ticker as ticker  # Import ticker module here

def millions_formatter(x, pos):
    return '{:.0f}M'.format(x * 1e-6)

def millions_formatter2(x, pos):
    return '{:.1f}M'.format(x * 1e-6)

def thousands_formatter(x, pos):
    return '{:.0f}K'.format(x * 1e-3)

def percentage_formatter(x, pos):
    return '{:.0%}'.format(x)

def custom_plot_all_scenarios(figureName, simulated_dfs_list, titles, baseline_index=0, num_rows=3, num_cols=2):
    num_scenarios = len(simulated_dfs_list)

    if num_rows < 3:
        plt.figure(figsize=(15, 7))
    else:
        plt.figure(figsize=(15, 10))

    label_list = ['Baseline (C0)', '20% testing (C1)', 'Delay in testing (C2)', '180% testing (C3)']
    cool_colors = ['#F24C3D', '#F2BE22', '#22A699', '#F29727']

    for i, title in enumerate(titles):

        plt.subplot(num_rows, num_cols, i + 1)

        for j, scenario_df in enumerate(simulated_dfs_list[1:], start=1):
            linestyle = '--' if j == baseline_index else '-'
            color = cool_colors[j - 1]
            variable_df = scenario_df[i]
            label = label_list[j]
            plt.plot(variable_df['Date'], variable_df['Mean'], label=label, color=color, linestyle=linestyle)
            plt.fill_between(variable_df['Date'], variable_df['Mean'] - np.sqrt(variable_df['Variance']),
                             variable_df['Mean'] + np.sqrt(variable_df['Variance']),
                             alpha=0.2, color=color)

        baseline_df = simulated_dfs_list[baseline_index][i]
        plt.plot(baseline_df['Date'], baseline_df['Mean'], label=f'Baseline (C{baseline_index})', color='black',
                 linestyle='--')

        plt.grid(True)
        plt.title(title)
        plt.xlabel('Date')
        if "Daily" in title:
            plt.ylabel('Daily Count')
        else:
            plt.ylabel('Cumulative Count')


        plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

        ax = plt.gca()
        if "Cumulative" in title:
            if "Cumulative Deaths" in title:
                ax.yaxis.set_major_formatter(ticker.FuncFormatter(millions_formatter2))  # Format y-axis labels in millions
            else:
                ax.yaxis.set_major_formatter(ticker.FuncFormatter(millions_formatter))  # Format y-axis labels in millions
        if "Daily" in title:
            if "Daily Diagnoses" in title:
                ax.yaxis.set_major_formatter(ticker.FuncFormatter(millions_formatter2))  # Format y-axis labels in millions
            else:
                ax.yaxis.set_major_formatter(ticker.FuncFormatter(thousands_formatter))  # Format y-axis labels in millions


    handles, labels = plt.gca().get_legend_handles_labels()
    plt.figlegend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=num_scenarios + 1)

    plt.tight_layout()
    plt.savefig(figureName, dpi=1000, bbox_inches="tight")
    plt.show()

# Example usage
titles = ['Cumulative Diagnoses', 'Daily Diagnoses', 'Cumulative Severe', 'Daily Severe', 'Cumulative Deaths', 'Daily Deaths']

# Simulated data for different scenarios including the baseline (C0)
simulated_dfs_C0 = make_df_list(plot_data_list_C0)
simulated_dfs_C1 = make_df_list(plot_data_list_C1)
simulated_dfs_C2 = make_df_list(plot_data_list_C2)
simulated_dfs_C3 = make_df_list(plot_data_list_C3)
simulated_dfs_C4 = make_df_list(plot_data_list_C4)

# Combine all scenarios into a list
# all_scenarios_list = [simulated_dfs_C0, simulated_dfs_C1, simulated_dfs_C2, simulated_dfs_C3, simulated_dfs_C4]
all_scenarios_list = [simulated_dfs_C0, simulated_dfs_C1, simulated_dfs_C2, simulated_dfs_C3]

# Plot all scenarios for each variable with variance
custom_plot_all_scenarios("Figure3_all", all_scenarios_list, titles)


In [ ]:
# Example usage
titles = ['Cumulative Hospitalizations', 'Daily Hospitalizations', 'Cumulative Deaths', 'Daily Deaths']

# Combine all scenarios into a list
# all_scenarios_list = [simulated_dfs_C0[2:], simulated_dfs_C1[2:], simulated_dfs_C2[2:], simulated_dfs_C3[2:], simulated_dfs_C4[2:]]
all_scenarios_list = [simulated_dfs_C0[2:], simulated_dfs_C1[2:], simulated_dfs_C2[2:], simulated_dfs_C3[2:]]

# Plot all scenarios for each variable with variance
custom_plot_all_scenarios("Figure3", all_scenarios_list, titles, num_rows = 2)


In [ ]:
# cuminf_df = [simulated_dfs_C0[0], simulated_dfs_C1[0], simulated_dfs_C2[0], simulated_dfs_C3[0], simulated_dfs_C4[0]]
# cumhos_df = [simulated_dfs_C0[2], simulated_dfs_C1[2], simulated_dfs_C2[2], simulated_dfs_C3[2], simulated_dfs_C4[2]]
# cumdead_df = [simulated_dfs_C0[4], simulated_dfs_C1[4], simulated_dfs_C2[4], simulated_dfs_C3[4], simulated_dfs_C4[4]]

def extract_cum_counts(startDateString = "2020-01-20", endDateString = "2022-12-31"):
    cuminf_df = [simulated_dfs_C0[0], simulated_dfs_C1[0], simulated_dfs_C2[0], simulated_dfs_C3[0]]
    cumhos_df = [simulated_dfs_C0[2], simulated_dfs_C1[2], simulated_dfs_C2[2], simulated_dfs_C3[2]]
    cumdead_df = [simulated_dfs_C0[4], simulated_dfs_C1[4], simulated_dfs_C2[4], simulated_dfs_C3[4]]


    cuminfdata = []; cumhospdata = []; cumdeaddata = []; cuminfdata_var = []; cumhospdata_var = []; cumdeaddata_var = []
    for i in range(len(cuminf_df)):

        cuminfdata_start = cuminf_df[i].Mean.loc[cuminf_df[i].Date == pd.to_datetime(startDateString)].values[0]
        cumhospdata_start = cumhos_df[i].Mean.loc[cumhos_df[i].Date == pd.to_datetime(startDateString)].values[0]
        cumdeaddata_start = cumdead_df[i].Mean.loc[cumdead_df[i].Date == pd.to_datetime(startDateString)].values[0]

        cuminfdata_end = cuminf_df[i].Mean.loc[cuminf_df[i].Date == pd.to_datetime(endDateString)].values[0]
        cumhospdata_end = cumhos_df[i].Mean.loc[cumhos_df[i].Date == pd.to_datetime(endDateString)].values[0]
        cumdeaddata_end = cumdead_df[i].Mean.loc[cumdead_df[i].Date == pd.to_datetime(endDateString)].values[0]

        cuminfdata_int = cuminfdata_end - cuminfdata_start 
        cumhospdata_int = cumhospdata_end - cumhospdata_start
        cumdeaddata_int = cumdeaddata_end - cumdeaddata_start

        cuminfdata.append(cuminfdata_int)
        cumhospdata.append(cumhospdata_int)
        cumdeaddata.append(cumdeaddata_int)

        cuminfdata_var.append(cuminf_df[i].Variance.loc[cuminf_df[i].Date == pd.to_datetime(endDateString)].values[0])
        cumhospdata_var.append(cumhos_df[i].Variance.loc[cumhos_df[i].Date == pd.to_datetime(endDateString)].values[0])
        cumdeaddata_var.append(cumdead_df[i].Variance.loc[cumdead_df[i].Date == pd.to_datetime(endDateString)].values[0])

    plotdict = {}
    plotdict['Mean Cumulative Infections'] = cuminfdata
    plotdict['Mean Cumulative Hospitalization'] = cumhospdata
    plotdict['Mean Cumulative Dead'] = cumdeaddata

    plotdict['Variance Cumulative Infections'] = cuminfdata_var
    plotdict['Variance Cumulative Hospitalization'] = cumhospdata_var
    plotdict['Variance Cumulative Dead'] = cumdeaddata_var
    plotdict['Scenario'] = ['Baseline (C0)', '20% testing (C1)', 'Delay in testing (C2)', '180% testing (C3)' ]
    plotdatadf = pd.DataFrame(plotdict)
    plotdatadf.to_csv(f"Cumulative Counts - {startDateString} to {endDateString}.csv", index=False)
    return plotdict

plotdict = extract_cum_counts(startDateString = "2020-01-20", endDateString="2022-12-31")

In [ ]:
pd.DataFrame(plotdict).to_csv("EntireStudyResults.csv")

In [ ]:
plotdict_winter2020 = extract_cum_counts(startDateString = "2020-10-01", endDateString="2021-03-01")
plotdict_winter2021 = extract_cum_counts(startDateString = "2021-10-01", endDateString="2022-03-01")

pd.DataFrame(plotdict_winter2020).to_csv("Winter2020.csv")
pd.DataFrame(plotdict_winter2021).to_csv("Winter2021.csv")

In [ ]:
# Extracting scenario labels
scenarios = plotdict['Scenario']

# Extracting plotdict for each category
infections = plotdict['Mean Cumulative Infections']
hospitalizations = plotdict['Mean Cumulative Hospitalization']
deaths = plotdict['Mean Cumulative Dead']

# Creating index array for scenarios
x = np.arange(len(scenarios))

# Bar width
bar_width = 0.25

# Plotting the bars
plt.bar(x - bar_width, infections, width=bar_width, label='Cumulative Infections')
plt.bar(x, hospitalizations, width=bar_width, label='Cumulative Hospitalization')
plt.bar(x + bar_width, deaths, width=bar_width, label='Cumulative Dead')

# Adding labels and title
plt.xlabel('Scenario')
plt.ylabel('Cumulative Counts')
plt.title('COVID-19 Simulation Results')
plt.xticks(x, scenarios)
plt.legend()

# Show the plot
plt.show()